# We import the libraries

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import .parent_selection as ps
import .crossover as cr
import logging

# Functions

In [17]:
def read_data(filepath):
    """ Read the data from the file and return the distance matrix and the list of cities positions """

    with open(filepath, 'r') as file:
        distance_matrix = False
        city_list = False
        for line in file:
            line = line.strip()
            # Create the distance matrix and the list of cities
            if line.startswith('DIMENSION'):
                dimension = int(line.split()[-1])
                cities = np.zeros((dimension, 2))
                distances = np.zeros((dimension, dimension))
                i=0
                j=0
            
            # What do we do with the line
            if line.startswith('EDGE_WEIGHT_SECTION'):
                distance_matrix = True
                continue
            elif line.startswith('DISPLAY_DATA_SECTION'):
                distance_matrix = False
                city_list = True
                continue
            elif line.startswith('EOF'):
                break
            
            # Save data
            if distance_matrix:
                distances[i,] = [int(x) for x in line.split()]
                i += 1
            if city_list:
                cities[j,] = line.split()[-2:]
                j += 1
        
        return distances, cities

In [ ]:
class TSP_Genetic:
    """Perform a Genetic Algorithm simulation for the TSP problem"""

    def __init__(
        self,
        generations=100,
        print_rate=10,
        m_rate=0.05,
        c_rate=0.8,
        select_parents=ps.tournament(tournament_size=3),
        crossover="OX1",
        mutation="inversion",
        elitism=0
    ):

        """Initialize the Genetic Algorithm simulation for the TSP problem.
            Args:
                generations: The number of generations to simulate in the genetic algorithm (100 by default).
                print_rate: How often to print the progress of the algorithm (Default: every 10 generations).
                m_rate: The mutation rate, representing the probability of mutation in offspring (Default: 0.05).
                c_rate: The crossover rate, representing the probability of crossover between parent chromosomes (Default: 0.8).
                select_parents: The method to select parents for crossover. Options are 'tournament' and 'roulette' (Default: 'tournament').
                crossover: The method to perform crossover between parent chromosomes.
                mutation: The method to perform mutation in offspring chromosomes.
                elitism: The number of best chromosomes to keep in the next generation (Default: 0).
        """
        self.generations = generations
        self.print_rate = print_rate

        self.m_rate = m_rate
        self.c_rate = c_rate

        self.select_parents = select_parents
        self.crossover = crossover
        self.mutation = mutation
        self.elitism = elitism

    def _fitness(self,x):
        """Calculate the fitness of a chromosome x for the TSP problem.
            Args:
                x: The chromosome to evaluate.
            Returns:
                The fitness of the chromosome.
        """
        fitness = 0
        for i in range(len(x))-1:
            fitness += self.distances[x[i],x[i+1]]
        fitness += self.distances[x[-1],x[0]]
        return fitness
    
    def fitness_population(self,population):
        """Calculate the fitness of a population of chromosomes for the TSP problem.
            Args:
                population: The population to evaluate.
            Returns:
                The fitness of each chromosome in the population.
        """
        return np.array([self._fitness(x) for x in population])
    
    def plot_route(self,z):
        """Plot the route of a chromosome z for the TSP problem.
            Args:
                z: The chromosome to plot.
        """
        # Plot the cities
        plt.plot(self.cities[:,0],self.cities[:,1],'o-')
        for (x, y), label in zip(self.cities, [str(i) for i in range(1, self.n_cities+1)]):
            plt.text(x, y + 0.1, label, ha='center', va='bottom')

        # Plot the route
        for i in range(len(z)-1):
            plt.plot([self.cities[z[i],0],self.cities[z[i+1],0]],[self.cities[z[i],1],self.cities[z[i+1],1]],'r-')

        plt.show()


    def run(self,population,cities,distances):
        """Run the Genetic Algorithm simulation for the TSP problem.
            Args:
                population: The initial population of chromosomes.
                cities: The list of positions of each city. A row represents a city and the first element its x coordinate and the second its y coordinate.
                distances: A matrix representing the distances between each pair of cities.
        """
        self.population = population
        self.population_size = self.population.shape[0]
        self.cities = np.array(cities)
        self.n_cities = self.cities.shape[0]
        self.distances = np.array(distances)

        # We need population_size-elitism to be even
        if (self.population_size-self.elitism) % 2 != 0:
            logging.warning("Population size - elitism is not even. Adding one to population size.")
            self.population_size += 1

        for generation in range(self.generations):
            # Get fitness of a population
            fitness = self.fitness(self.population)
            offspring = []
            #Elitism
            if self.elitism > 0:
                offspring.append(self.population[np.argsort(fitness)[:self.elitism]])
            
            # Generate offspring:
            for i in range((self.population_size-self.elitism)/2):
                # Select parents
                parent1= self.select_parents(population,fitness)
                parent2= self.select_parents(population,fitness)
                
                #Crossover
                if random.random() < self.c_rate:
                    # Generate offspring
                    crossover= cr.Crossover(parent1,parent2)
                    child1, child2 = getattr(crossover, self.crossover)()
                else:
                    child1, child2 = parent1, parent2

                # Mutate offspring
                # TODO: Write the mutation function
                if random.random() < self.m_rate:
                    child1 = self.mutation(child1)
                if random.random() < self.m_rate:
                    child2 = self.mutation(child2)

                offspring.append(child1,child2)

            population = np.array(offspring)
            # Print progress
            if generation-1 % self.print_rate == 0:
                logging.INFO(f"Generation {generation} - Best cromosome: {population[np.argmin(fitness)]}, Best fitness: {np.min(fitness)}")
                self.plot_route(population[np.argmin(fitness)])
        
        return self.population[0]

In [18]:
distances,cities= read_data('bays29.tsp')
    